In [2]:
import pandas as pd
import sqlite3
from helper import load_dotenv

In [3]:
config = load_dotenv()

In [4]:
con = sqlite3.connect( config['DB_PATH'] )

In [12]:
data = pd.read_sql(
    '''
        SELECT rfvps.*, ifnull(rffc.fc_available,0) AS fc_available FROM (
            SELECT variant_name, pdf_filename, job, 1 AS "fc_available" FROM related_file rf 
            WHERE "type" = "4c"
        ) rffc
        FULL OUTER JOIN (
            SELECT variant_name, pdf_filename, job, count(*) >= 1 AS vps_available FROM related_file rf 
            WHERE "type" IN ("C","M","Y","K")
            GROUP BY variant_name, pdf_filename, job
        ) rfvps
        ON rffc.variant_name = rfvps.variant_name AND rffc.pdf_filename = rfvps.pdf_filename AND rffc.job = rfvps.job
    ''',
    con
)
data.vps_available = data.vps_available == 1
data.fc_available = data.fc_available == 1

In [15]:
data.fc_available.value_counts()

True     1225
False     181
Name: fc_available, dtype: int64